In [12]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_excel("D:/Browns/Web Scrapping/top_5_motorbikes.xlsx")

# Clean dataframe: convert NaN -> None
df_cleaned = df.where(pd.notnull(df), None)

data_json = df_cleaned.to_dict(orient="records")


In [4]:
#data_json

In [10]:
import os
from dotenv import load_dotenv
from azure.cosmos import CosmosClient, PartitionKey
from datetime import datetime, timedelta

# Load Cosmos credentials from .env
load_dotenv()
cosmos_endpoint = os.getenv("cosmos_e")
cosmos_key = os.getenv("cosmos_k")

# Check if the environment variables are loaded correctly
if not cosmos_endpoint or not cosmos_key:
    raise ValueError("Cosmos endpoint or key is missing. Please check your .env file.")

# Cosmos setup
client = CosmosClient(cosmos_endpoint, cosmos_key)
database_name = "sachithws"

# Create database if not exists
db = client.create_database_if_not_exists(id=database_name)

# Create container with next day's date as name
next_date = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
container_name = next_date

container = db.create_container_if_not_exists(
    id=container_name,
    partition_key=PartitionKey(path="/id"),
    
)

In [14]:

# Step 2: Insert to Cosmos DB
for idx, item in enumerate(data_json):
    item['id'] = str(idx + 1)  # Required by Cosmos DB
    if pd.isna(item.get('Kilometers')):
        item['Kilometers'] = None
    container.upsert_item(item)
    print(f" Uploaded record {idx + 1}")




 Uploaded record 1
 Uploaded record 2
 Uploaded record 3
 Uploaded record 4
 Uploaded record 5
